In [1]:
import numpy as np
import scipy
import utils
from time import time
from utils import HouseHolder, QR, SVD

%load_ext autoreload
%autoreload 2

# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [2]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64).T


# A = np.array([[1, 0, 1],
#               [2, 5**.5, 0],
#               [0, 0, 1],
#               [0, 0, 1]])
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 0.  2.  0.  0.  0.  0.  0.]
 [ 0. -5.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [3]:
print(Qt @ A @ P)

[[ 0.  2.  0.  0.  0.  0.  0.]
 [ 0. -5.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


## Phase-II

In [4]:
m = n = 500
A = np.random.rand(m,n)

**Test Fast Multiplication for A@B, where B is upper bidiagonal matrix.**

In [5]:
B, _, _ = SVD.svd_phaseI(A)

In [6]:
numpy_mul_begin = time()
for i in range(2000):
    A@B
numpy_mul_end = time()
numpy_mul_end - numpy_mul_begin

4.825413227081299

In [7]:
fastMul_begin = time()
for i in range(2000):
    SVD.fastMult_upper_bidiagonal(A, B)
fastMul_end = time()
fastMul_end - fastMul_begin

6.074538469314575

**Test SVD**

Choose the parameter phaseII as 'A', 'B1', 'B2' to test different implementations of phase II

In [33]:
m = 1024
n = 1024
A = np.random.rand(m,n)

In [36]:
U, S, Vt = SVD.svd(A, phaseII='A')
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

phaseI: 11.507385s
phaseII: 58.388663s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %


**Accuracy Test:**

In [10]:
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %


**Scipy SVD**

In [11]:
U, S, Vt  = scipy.linalg.svd(A, full_matrices=False)
# print(np.abs(U@np.diag(S)@Vt - A))
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")
# U, S, Vt

Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %
